In [1]:
# !pip install dashscope
import json
import os
import dashscope
from dashscope.api_entities.dashscope_response import Role


In [2]:
# obtain DASHSCOPE_API_KEY from environment variable
api_key = os.environ.get('DASHSCOPE_API_KEY')
dashscope.api_key = 'sk-16f2c80d9d3e4d7d9cc2f6b43f22dfbd' # https://bailian.console.aliyun.com/?tab=model#/api-key
# dashscope.api_key ='sk-qzkddkwofxndanqzonvabhwqbpwganpxkuuoghtxgvmzygnt' # https://cloud.siliconflow.cn/me/account/ak

## 1 emotion analysis

In [ ]:

# encapsulate model response function
def get_response(messages):
    response = dashscope.Generation.call(
        model = 'deepseek-v3',
        messages=messages,
        result_format='message' # return full message object
    )
    return response

# test the function
review = 'This product has excellent sound quality that exceeds expectations.'

messages=[
    {"role": "system", "content": "You are a sentiment analyst. Help me determine whether the product review is positive or negative. Reply with a single word: Positive or Negative."},
    {"role": "user", "content": review}
]

response = get_response(messages)
response.output.choices[0].message.content

## 2 Weather Function

In [ ]:
# Getting Weather information
def get_current_weather(location, unit="Celsius"):
    temperature = -1
    if '北京' in location or 'Beijing' in location:
        temperature = 10
    if '上海' in location or 'Shanghai' in location:
        temperature = 15
    if '深圳' in location or 'Shenzhen' in location:
        temperature = 20

    weather_info = {
        "location": location,
        "temperature": temperature,
        "unit": unit,
        "description": "Partly cloudy",
        "forecast": [
            {"day": "Monday", "description": "Sunny", "high": temperature + 2, "low": temperature - 2},
            {"day": "Tuesday", "description": "Rainy", "high": temperature + 1, "low": temperature - 3},
            {"day": "Wednesday", "description": "Cloudy", "high": temperature + 3, "low": temperature - 1},
        ]
    }
    
    return json.dumps(weather_info)


# Encapsulate the weather function
def get_response(messages):
    try:
        response = dashscope.Generation.call(
            # model = 'deepseek-v3',
            model='qwen-max',
            messages=messages,
            functions=functions,
            result_format='message' # return full message object
        )
        return response
    except Exception as e:
        print(f"Error occurred: {e}")
        return None


# Test the weather function with Q&A
def run_conversation():
    query = "What's the weather like in Beijing today?"
    messages = [{"role":"user", "content": query}]
    response = get_response(messages)
    
    # step1 1st response from model
    if not response or not response.output:
        print("Failed to get response from model.")
        return None
    
    print('response=', response)

    message = response.output.choices[0].message
    messages.append(message) # keep Historical conversations 保留对话历史，保持上下文
    print("messages=", message)

    # Step2 User Call Function 
    if hasattr(message, 'function_call') and message.function_call:
        function_call = message.function_call
        # tool_name = function_call.name
        tool_name = function_call['name']

    # step3 Excute Function
        arguments = json.loads(function_call['arguments'])
        print("arguments=", arguments)
        tool_response = get_current_weather(
            location=arguments.get('location'),
            unit=arguments.get('unit', 'Celsius'),
        )

        tool_info = {
            "role": "function",
            "name": tool_name,
            "content": tool_response
        }
        messages.append(tool_info)
        print("messages after tool response=", messages)

        # step4 obatain 2nd response from model
        response = get_response(messages)
        if not response or not response.output:
            print("Failed to get 2nd response from model after function call.")
            return None

        print('response=', response)
        message = response.output.choices[0].message
        return message
    return message

# Description of the location
functions = [
    {
        "name": "get_current_weather", # 要用哪个工具
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {
                    "type": "string",
                    "enum": ["Celsius", "Fahrenheit"],
                    "description": "The unit of temperature",
                },
            },
            "required": ["location"],
        },
    }
]

if __name__ == "__main__":
    final_message = run_conversation()
    if final_message:
        print("Final response from model:", final_message)
    else:
        print("No final response obtained.")

## 3 Table Extraction

In [ ]:
# encapsulate model response function
def get_response(messages):
    response = dashscope.MultiModalConversation.call(
        model = 'qwen-v1-plus',
        messages=messages
    )
    return response

content = [
    {'image': ''},
    {'text': 'This is a picture of a table. Please exract the contents on the table and return a JSON file.'}
]


